In [2]:
import pandas as pd
df = pd.read_csv("/home/chanbo.s/personalized_ecoli/microbiologyevents.csv")
micro_df = df[df["org_name"] == "ESCHERICHIA COLI"]
len(micro_df)
patient_df = pd.read_csv("/home/chanbo.s/personalized_ecoli/patients.csv")
patient_df = patient_df.drop(['anchor_year', 'anchor_year_group', 'dod'], axis=1)
print(patient_df.head())



agg_micro = micro_df.groupby('subject_id').agg({
    'ab_name': list,
    'interpretation': list,
    'spec_type_desc': lambda x: x.mode()[0],  
    'test_name': lambda x: x.mode()[0]        
}).reset_index()


def categorize_antibiotics(row):
    effective, ineffective = set(), set()
    for ab, interp in zip(row['ab_name'], row['interpretation']):
        if interp == 'S':
            effective.add(ab)
        elif interp in {'R', 'I'}:
            ineffective.add(ab)
    return list(effective), list(ineffective)


agg_micro[['effective_antibiotics', 'ineffective_antibiotics']] = agg_micro.apply(
    categorize_antibiotics, axis=1, result_type='expand'
)


agg_micro['num_antibiotics_tested'] = agg_micro['ab_name'].apply(lambda x: len(set(x)))


patient_df = patient_df.drop(columns=['anchor_year', 'anchor_year_group', 'dod'], errors='ignore')
merged_df = agg_micro.merge(patient_df, on='subject_id', how='left')


antibiotic_counts = pd.Series([ab for sublist in merged_df['effective_antibiotics'] for ab in sublist]).value_counts()
common_antibiotics = set(antibiotic_counts[antibiotic_counts >= 5].index)


merged_df['effective_antibiotics'] = merged_df['effective_antibiotics'].apply(
    lambda lst: [ab for ab in lst if ab in common_antibiotics]
)
merged_df['ineffective_antibiotics'] = merged_df['ineffective_antibiotics'].apply(
    lambda lst: [ab for ab in lst if ab in common_antibiotics]
)
merged_df = merged_df[merged_df['effective_antibiotics'].apply(len) > 0]
merged_df.to_csv("merged_microbiology_admissions_final.csv.csv", index=False)
print("Merged and cleaned dataset saved as 'merged_microbiology_admissions_final.csv'.")


/tmp/ipykernel_276416/545082658.py:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/chanbo.s/personalized_ecoli/microbiologyevents.csv")


   subject_id gender  anchor_age
0    10000032      F          52
1    10000048      F          23
2    10000058      F          33
3    10000068      F          19
4    10000084      M          72
Merged and cleaned dataset saved as 'merged_microbiology_admissions_final.csv'.
